In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [ ]:
np.random.seed(171)

# 演習1

## ダミーデータを作ります

In [ ]:
# ダミーデータを生成する関数
def generate_2dim_normal(mean, variance, covariance, sample_size):
    cov = [[variance,covariance],
           [covariance,variance]]
    return np.random.multivariate_normal(mean, cov, sample_size)

In [ ]:
cluster1 = generate_2dim_normal(mean = [0, 8], variance=1, covariance=0, sample_size=500)
cluster2 = generate_2dim_normal(mean = [-1, 0], variance=1, covariance=0, sample_size=500)
cluster3 = generate_2dim_normal(mean = [10, 10], variance=1, covariance=0, sample_size=300)
cluster4 = generate_2dim_normal(mean = [5, 5.5], variance=0.8, covariance=-0.1, sample_size=200)
data = np.vstack((cluster1, cluster2, cluster3, cluster4))

データを観察してみましょう。

In [ ]:
data

In [ ]:
df = pd.DataFrame(data, columns=["x", "y"])

In [ ]:
df.plot(kind="scatter", x="x", y="y")

## クラスタリングしてみましょう

In [ ]:
from sklearn.cluster import KMeans

このデータは4つのクラスタを混ぜて作られているので、4つのクラスタに分けてみましょう。ちゃんと分けられるでしょうか？

In [ ]:
d_model = #TODO KMeansのモデルを作ってください。

In [ ]:
d_clusters = # TODO モデルをデータから学習してください。

In [ ]:
data.shape

In [ ]:
df = pd.DataFrame(np.c_[data, d_clusters], columns=["x", "y", "cluster"])

In [ ]:
colors = ["red", "green", "blue", "yellow"]

In [ ]:
fig, ax = plt.subplots(1, 1)
for i, (_, group) in enumerate(df.groupby("cluster")):
    group.plot(kind="scatter", x="x", y="y", color=colors[i], ax=ax)

# 演習2

## データを読み込みます

In [ ]:
rating = pd.read_csv("data/rating.csv.gz")
anime = pd.read_csv("data/anime.csv")

## データを観察します

In [ ]:
anime.head()

In [ ]:
rating.head()

In [ ]:
rating.tail()

In [ ]:
rating.shape

In [ ]:
anime.shape

## 前処理

行にアニメ、列にユーザーをとる特徴ベクトルを作ります。
出来上がったものは行列になり、行列要素の各値は0か1の値を取ります。

i行目 j列目の値が1のとき、「アニメID=iのアニメをユーザーID=jのユーザーが見た」という意味になります。

In [ ]:
from scipy.sparse import csr_matrix

### 特徴量ベクトルの作成

行にアニメIDを取ります。

In [ ]:
row = rating["anime_id"]

列にユーザーIDを取ります。

In [ ]:
column = rating["user_id"]

ユーザーがアニメを見たことを表すベクトルを用意します。

In [ ]:
value = np.ones(rating.shape[0], dtype=np.float)

In [ ]:
value

以上で、row、column、valueの各位置が特徴ベクトルの情報に対応します。
例えば、row[10]行目のcolumn[10]列目の値は、value[10]になります。

In [ ]:
row[10], column[10], value[10]

今回は936行目、1列目の値が1になるという意味です。

特徴ベクトルを作ります。

In [ ]:
features = csr_matrix((value, (row, column)), dtype=np.float)

先頭10行を見てみましょう。

In [ ]:
features[:10].todense()

In [ ]:
features.shape

視聴人数が10人未満のアニメに限定します。視聴人数1人以下のデータも除外しましょう。

In [ ]:
min_view = 2
max_view = 10

まず、そのようなデータがいくつあるのかを調べます。

In [ ]:
row_map = np.logical_and(min_view <= features.sum(axis=1), features.sum(axis=1) < max_view)

In [ ]:
row_map = np.array(row_map).flatten()

In [ ]:
row_map.sum()

10人未満の行だけ取り出しましょう。

In [ ]:
features_shrink_row = features[row_map]

これで視聴人数が10回以下のデータが取り出せました。行列の形を見てみましょう。

In [ ]:
features_shrink_row.shape

行数が減っていますね。

同様に、視聴回数が0になっているユーザーも除外しましょう。
(行を削っているので、削られた行しか見ていなかったユーザーも除外されるはずです。)

まず、視聴回数が1回以上のユーザーの人数を数えてみましょう。

In [ ]:
column_map = 0 < features_shrink_row.sum(axis=0)

In [ ]:
column_map = np.array(column_map).flatten()

In [ ]:
column_map.sum()

視聴回数が1回以上のユーザーを取り出します。

In [ ]:
features_compact = features_shrink_row[:, column_map]

ちゃんと取り出せているか確認しましょう。

In [ ]:
features_compact.shape

これぐらいの大きさであればメモリに乗ると思いますので、dense arrayに変換します。

In [ ]:
features_compact = features_compact.todense()

これで特徴量ベクトルが完成しました。お疲れ様でした。

### 特徴量ベクトルとアニメIDの対応をとる

視聴人数でベクトルの数を削ったため、行列の行番号とアニメIDの対応が崩れてしまいました。そこで、対応をとるための辞書を作成します。

上記で定義したrow_mapには、削る前の行列で、削る行にFalse, 残る行にTrueが入っています。

In [ ]:
row_map

Trueが出てくる位置が最終的に残ったアニメIDなので、対応表は以下のように作ることができます。

In [ ]:
row_id_to_anime_id = []
for anime_id, b in enumerate(row_map):
    if b:
        row_id_to_anime_id.append(anime_id)

試しに10行目のアニメIDを見てみましょう。

In [ ]:
row_id_to_anime_id[10]

### データを正規化

0/1ベクトルをそのまま使うとよくないので、各ベクトルの大きさを1にします。  
(長さが1のベクトルに対してユークリッド距離を計算すると、コサイン距離と一致することを狙っています。)

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
n = Normalizer(norm="l2")
f = n.fit_transform(features_compact)

## 出来上がった特徴ベクトルを可視化しよう。

今回作成したベクトルは、2057次元もあるので、通常の方法では目視できません。主成分分析(PCA)を使って可視化してみましょう。

In [ ]:
from sklearn.decomposition import PCA

3次元に圧縮します。

余談ですが、%%timeをセルの先頭に書いておくと、所要時間を計ることができます。

In [ ]:
%%time

pca = # TODO PCAのモデルを作ってください。
comp = # TODO 圧縮してください。(fit_transform)

これで、変数compに圧縮結果が入りました。みてみましょう。

In [ ]:
comp

プロットもしてみます。

まず、データフレームにしておきます。

In [ ]:
comp_df = pd.DataFrame(comp, columns=["x", "y", "z"])

X, Y軸からみてみましょう。

In [ ]:
comp_df.plot(kind="scatter", x="x", y="y")

続いてY, Z軸。

In [ ]:
comp_df.plot(kind="scatter", x="z", y="y")

最後に、X, Z軸でも見ます。

In [ ]:
comp_df.plot(kind="scatter", x="x", y="z")

なんとなく塊が見えましたね。では、この塊をクラスタリングしましょう。

## クラスタリング

In [ ]:
from sklearn.cluster import KMeans

### 分割数を決める

幾つの塊に分ければいいのか検討しましょう。PCAの結果を見る限り、4〜8くらいでしょうか。

判断の参考にするために、エルボー法を使います。

In [ ]:
seed=131
inertias = dict(
    n_clusters=[],
    inertia=[],
)
for i in range(2, 11):
    model = # TODO i個のクラスタに分けてください。
    model.fit(f)
    inertias["n_clusters"].append(i)
    inertias["inertia"].append(model.inertia_)
    print("n_clusters={} SSE={}".format(i, model.inertia_))

In [ ]:
pd.DataFrame(inertias["inertia"], index=inertias["n_clusters"]).plot(legend=False)

### 学習

難しいところですが、上のグラフを参考にしつつクラスタ数を決めます。

In [ ]:
n_clusters = # TODO クラスタ数を決めます

決めたクラスタ数でモデルを学習しましょう。

In [ ]:
model = KMeans(n_clusters, random_state=seed)

In [ ]:
clusters = model.fit_predict(f)

### 結果をプロットしましょう。

カラーマップを作ります。

In [ ]:
cmap = plt.get_cmap("rainbow")
colors = [cmap(c / float(n_clusters)) for c in range(n_clusters)]

In [ ]:
df = pd.DataFrame(np.c_[comp, clusters], columns=["x", "y", "z", "cluster"])

X,Y座標を見ましょう。

In [ ]:
fig, ax = plt.subplots(1, 1)
for i, (key, group) in enumerate(df.groupby("cluster")):
    group.plot(kind="scatter", x="x", y="y", color=colors[i], ax=ax)

Y,Z座標を見ましょう。

In [ ]:
fig, ax = plt.subplots(1, 1)
for i, (key, group) in enumerate(df.groupby("cluster")):
    group.plot(kind="scatter", x="z", y="y", color=colors[i], ax=ax)

X, Z座標を見ましょう。

In [ ]:
fig, ax = plt.subplots(1, 1)
for i, (key, group) in enumerate(df.groupby("cluster")):
    group.plot(kind="scatter", x="x", y="z", color=colors[i], ax=ax)

## 分析

得られたクラスタがどんなクラスタなのか確認しましょう。

クラスタ中心はcluster_centers_に格納されています。

In [ ]:
model.cluster_centers_

In [ ]:
model.cluster_centers_.shape

### 距離を計算します

In [ ]:
f.shape

In [ ]:
from sklearn.metrics import euclidean_distances

In [ ]:
dists = euclidean_distances(f, model.cluster_centers_)

In [ ]:
dists.shape

In [ ]:
dists

### クラスタ中心を確認

各クラスタの最も中心に近いアニメはなんでしょうか?

クラスタ0の中心に近いアニメを確認しましょう。

argsortを使うと、データの小さい順に並び替えて、データの行番号を返してくれます。

In [ ]:
cluster0 = dists[:, 0].argsort()[clusters == 0][:10]
cluster0

In [ ]:
for row_id in cluster0:
    print(dists[row_id, 0], anime[anime["anime_id"] == row_id_to_anime_id[row_id]]["name"].tolist()[0])

同じ要領で他のクラスタも確認します。

In [ ]:
for i in range(n_clusters):
    print("*** Cluster {} ***".format(i))
    nearest = dists[:, i].argsort()[clusters == i][:10]
    for row_id in nearest:
        print(dists[row_id, i], anime[anime["anime_id"] == row_id_to_anime_id[row_id]]["name"].tolist()[0])
    print()

## グループワーク

得られたクラスタの意味を解釈してみましょう。